In [1]:
# upload fetal.csv file here in content directory.
!ls
!pwd

fetal_health.csv  sample_data
/content


 # <h1 style='background:#F7B2B0; border:0; color:black'><center>FETAL HEALTH</center></h1> 
 

 
Cardiotocography (CTG) is used during pregnancy to monitor fetal heart rate and uterine contractions. It is monitor fetal well-being and allows early detection of fetal distress. 

 CTG interpretation helps in determining if the pregnancy is high or low risk.  An abnormal CTG may indicate the need for further investigations and potential intervention.

In this project, I will create a model to classify the outcome of Cardiotocogram test to ensure the well being of the fetus. 

 # **<span style="color:#F7B2B0;">TABLE OF CONTENTS</span>**

**IMPORTING LIBRARIES**

**LOADING DATA**

**DATA PREPROCESSING**

**DATA ANALYSIS**

**MODEL BUILDING**

**CONCLUSIONS**

 # **<span style="color:#F7B2B0;">IMPORTING LIBRARIES</span>**

In [2]:
# Cell:1
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score

import statsmodels.api as sm
import tensorflow as tf
from tensorflow import keras
np.random.seed(0)

# **<span style="color:#F7B2B0;">LOADING DATA</span>**

In [3]:
# Cell:2
data = pd.read_csv("./fetal_health.csv")
data.head()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.0,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.0,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.0,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0


In [4]:
# Cell:3
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126 entries, 0 to 2125
Data columns (total 22 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   baseline value                                          2126 non-null   float64
 1   accelerations                                           2126 non-null   float64
 2   fetal_movement                                          2126 non-null   float64
 3   uterine_contractions                                    2126 non-null   float64
 4   light_decelerations                                     2126 non-null   float64
 5   severe_decelerations                                    2126 non-null   float64
 6   prolongued_decelerations                                2126 non-null   float64
 7   abnormal_short_term_variability                         2126 non-null   float64
 8   mean_value_of_short_term_variability  

In [5]:
# Cell:4
data.describe().T

,count,mean,std,min,25%,50%,75%,max
baseline value,2126.0,133.303857,9.840844,106.0,126.000,133.000,140.000,160.000
accelerations,2126.0,0.003178,0.003866,0.0,0.000,0.002,0.006,0.019
fetal_movement,2126.0,0.009481,0.046666,0.0,0.000,0.000,0.003,0.481
uterine_contractions,2126.0,0.004366,0.002946,0.0,0.002,0.004,0.007,0.015
light_decelerations,2126.0,0.001889,0.002960,0.0,0.000,0.000,0.003,0.015
severe_decelerations,2126.0,0.000003,0.000057,0.0,0.000,0.000,0.000,0.001
prolongued_decelerations,2126.0,0.000159,0.000590,0.0,0.000,0.000,0.000,0.005
abnormal_short_term_variability,2126.0,46.990122,17.192814,12.0,32.000,49.000,61.000,87.000
mean_value_of_short_term_variability,2126.0,1.332785,0.883241,0.2,0.700,1.200,1.700,7.000
percentage_of_time_with_abnormal_long_term_variability,2126.0,9.846660,18.396880,0.0,0.000,0.000,11.000,91.000


**On This Dataset**
**Cardiotocograms (CTGs)** are a simple and cost accessible option to assess fetal health, allowing healthcare professionals to take action in order to prevent child and maternal mortality. The equipment itself works by sending ultrasound pulses and reading its response, thus shedding light on fetal heart rate (FHR), fetal movements, uterine contractions and more.

This dataset contains 2126 records of features extracted from Cardiotocogram exams, which were then classified by expert obstetrician into 3 classes:

1. Normal
1. Suspect
1. Pathological

**Features**

* **'baseline value'** FHR baseline (beats per minute)
* **'accelerations'** Number of accelerations per second
* **'fetal_movement'** Number of fetal movements per second
* **'uterine_contractions'** Number of uterine contractions per second
* **'light_decelerations'** Number of light decelerations per second
* **'severe_decelerations'** Number of severe decelerations per second
* **'prolongued_decelerations'** Number of prolonged decelerations per second
* **'abnormal_short_term_variability'** Percentage of time with abnormal short term variability
* **'mean_value_of_short_term_variability'** Mean value of short term variability
* **'percentage_of_time_with_abnormal_long_term_variability'** Percentage of time with abnormal long term variability
* **'mean_value_of_long_term_variability'** Mean value of long term variability 
* **'histogram_width'** Width of FHR histogram
* **'histogram_min' Minimum** (low frequency) of FHR histogram
* **'histogram_max' Maximum** (high frequency) of FHR histogram
* **'histogram_number_of_peaks'** Number of histogram peaks
* **'histogram_number_of_zeroes'** Number of histogram zeros
* **'histogram_mode'** Histogram mode
* **'histogram_mean'** Histogram mean
* **'histogram_median'** Histogram median
* **'histogram_variance'** Histogram variance
* **'histogram_tendency'** Histogram tendency

* **Target**
* **'fetal_health'** Tagged as 1 (Normal), 2 (Suspect) and 3 (Pathological)

# **<span style="color:#F7B2B0;">EXPLORATORY DATA ANALYSIS (EDA)</span>**

**The analysis consist of:**
* Feature exploration
* Null values removal
* Feature normaliztion/ standardization
* Feature engineering
* Feature selection


In [ ]:
# Cell:5
# Visually Check regression of each data column on the class (fetal_heath)

y = data['fetal_health']
x1 = data[['baseline value', 'accelerations', 'fetal_movement', 'uterine_contractions','light_decelerations', 'severe_decelerations', 'prolongued_decelerations',
          'abnormal_short_term_variability', 'mean_value_of_short_term_variability', 'percentage_of_time_with_abnormal_long_term_variability',
          'mean_value_of_long_term_variability', 'histogram_width', 'histogram_min', 'histogram_max', 'histogram_number_of_peaks', 'histogram_number_of_zeroes',
          'histogram_mode', 'histogram_mean', 'histogram_median', 'histogram_variance', 'histogram_tendency']]

# plt.scatter(x1, y)
# plt.xlabel('baseline value', fontsize=20)
# plt.ylabel('fetal_health', fontsize=20)
# plt.show()


Using Stats model API to run reression analysis on each feature with target to know which features are important and have most effect on the target classification.

In [7]:
# Cell:6
#check regression of each feature on target class
x = sm.add_constant(x1)
results = sm.OLS(y, x).fit()
results.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           fetal_health   R-squared:                       0.607
Model:                            OLS   Adj. R-squared:                  0.604
Method:                 Least Squares   F-statistic:                     162.8
Date:                Sun, 03 Apr 2022   Prob (F-statistic):               0.00
Time:                        11:21:34   Log-Likelihood:                -986.93
No. Observations:                2126   AIC:                             2016.
Df Residuals:                    2105   BIC:                             2135.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================
                                                             coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
const                                                      0.2620      0.135      1.937      0.053      -0.003       0.527
baseline value                                             0.0155      0.002      6.995      0.000       0.011       0.020
accelerations                                              0.1378      3.690      0.037      0.970      -7.099       7.375
fetal_movement                                             0.0842      0.192      0.439      0.661      -0.292       0.460
uterine_contractions                                     -22.8929      3.226     -7.096      0.000     -29.220     -16.566
light_decelerations                                       -3.4471      5.267     -0.654      0.513     -13.776       6.882
severe_decelerations                                     761.1237    156.006      4.879      0.000     455.182    1067.065
prolongued_decelerations                                 362.2050     23.525     15.397      0.000     316.071     408.339
abnormal_short_term_variability                            0.0086      0.001     12.716      0.000       0.007       0.010
mean_value_of_short_term_variability                       0.0017      0.016      0.102      0.919      -0.030       0.034
percentage_of_time_with_abnormal_long_term_variability     0.0117      0.001     18.742      0.000       0.010       0.013
mean_value_of_long_term_variability                        0.0058      0.002      2.697      0.007       0.002       0.010
histogram_width                                            0.0002      0.000      0.619      0.536      -0.000       0.001
histogram_min                                              0.0034      0.001      5.653      0.000       0.002       0.005
histogram_max                                              0.0036      0.001      5.106      0.000       0.002       0.005
histogram_number_of_peaks                                 -0.0029      0.004     -0.669      0.504      -0.012       0.006
histogram_number_of_zeroes                                 0.0124      0.013      0.958      0.338      -0.013       0.038
histogram_mode                                            -0.0037      0.002     -2.405      0.016      -0.007      -0.001
histogram_mean                                            -0.0035      0.002     -1.456      0.146      -0.008       0.001
histogram_median                                          -0.0112      0.003     -3.724      0.000      -0.017      -0.005
histogram_variance                                         0.0028      0.000      5.988      0.000       0.002       0.004
histogram_tendency                                         0.0825      0.023      3.537      0.000       0.037       0.128
========

All the features having P-Value less than 0.05 can be considered as important and the rest columns can be dropped.


In [8]:
important_features = ['baseline value', 'accelerations', 'uterine_contractions', 'severe_decelerations', 'prolongued_decelerations', 'abnormal_short_term_variability', 'mean_value_of_long_term_variability', 'histogram_min', 'histogram_max'
'histogram_mode', 'histogram_median', 'histogram_variance', 'histogram_tendency']

# **<span style="color:#F7B2B0;">MODEL SELECTION AND BUILDING</span>**
In this section we will:
* Set up features(X) and target(Y)
* Scale the features
* Split training and test sets 
* Model selection
* Hyperparameter tuning

In [9]:
# Cell:7
#assigning values to features as X and target as y
X=data.drop(["fetal_health",'accelerations', 'fetal_movement', 'light_decelerations', 'mean_value_of_short_term_variability',
             'histogram_width', 'histogram_number_of_peaks', 'histogram_number_of_zeroes', 'histogram_mean'],axis=1)
y=data["fetal_health"]

#Set up a standard scaler for the features
col_names = list(X.columns)
s_scaler = preprocessing.StandardScaler()
X_df= s_scaler.fit_transform(X)
X_df = pd.DataFrame(X_df, columns=col_names)   
X_df.describe().T
X_df.head()

,count,mean,std,min,25%,50%,75%,max
baseline value,2126.0,4.161509e-16,1.000235,-2.775197,-0.742373,-0.030884,0.680604,2.713428
uterine_contractions,2126.0,-2.625683e-15,1.000235,-1.482465,-0.803434,-0.124404,0.894142,3.610264
severe_decelerations,2126.0,7.254767e-16,1.000235,-0.057476,-0.057476,-0.057476,-0.057476,17.398686
prolongued_decelerations,2126.0,4.324961e-15,1.000235,-0.268754,-0.268754,-0.268754,-0.268754,8.208570
abnormal_short_term_variability,2126.0,2.687304e-16,1.000235,-2.035639,-0.872088,0.116930,0.815060,2.327675
percentage_of_time_with_abnormal_long_term_variability,2126.0,2.291362e-15,1.000235,-0.535361,-0.535361,-0.535361,0.062707,4.412293
mean_value_of_long_term_variability,2126.0,1.029959e-15,1.000235,-1.455081,-0.637583,-0.139975,0.464263,7.555172
histogram_min,2126.0,-6.120326e-17,1.000235,-1.474609,-0.899376,-0.019608,0.893996,2.213648
histogram_max,2126.0,-4.047144e-17,1.000235,-2.342558,-0.670314,-0.112899,0.555999,4.123453
histogram_mode,2126.0,6.141215e-17,1.000235,-4.729191,-0.516077,0.094519,0.644055,3.025381


In [ ]:
# Cell:8
#looking at the scaled features
shades =["#f7b2b0","#c98ea6","#8f7198","#50587f", "#003f5c"]
plt.figure(figsize=(20,10))
sns.boxenplot(data = X_df,palette = shades)
plt.xticks(rotation=90)
plt.show()

In [12]:
# Cell:9
#spliting test and training sets
X_train, X_test, y_train,y_test = train_test_split(X_df,y,test_size=0.15,random_state=42)

In [15]:
# Cell:10
#A quick model selection process
#pipelines of models( it is short was to fit and pred)
pipeline_lr=Pipeline([('lr_classifier',LogisticRegression(random_state=10))])

pipeline_dt=Pipeline([ ('dt_classifier',DecisionTreeClassifier(random_state=42))])

pipeline_rf=Pipeline([('rf_classifier',RandomForestClassifier())])

pipeline_svc=Pipeline([('sv_classifier',SVC())])

pipeline_knn=Pipeline([('knn_classifier',KNeighborsClassifier(n_neighbors=10,p=4,metric='euclidean'))])


# List of all the pipelines
pipelines = [pipeline_lr, pipeline_dt, pipeline_rf, pipeline_svc, pipeline_knn]

# Dictionary of pipelines and classifier types for ease of reference
pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: 'RandomForest', 3: "SVC", 4: 'KNN'}


# Fit the pipelines
for pipe in pipelines:
    pipe.fit(X_train, y_train)

#cross validation on accuracy 
cv_results_accuracy = []
for i, model in enumerate(pipelines):
    cv_score = cross_val_score(model, X_train,y_train, cv=10 )
    cv_results_accuracy.append(cv_score)
    print("%s: %f " % (pipe_dict[i], cv_score.mean()))


Logistic Regression: 0.885991 
Decision Tree: 0.915890 
RandomForest: 0.937486 
SVC: 0.901489 
KNN: 0.897618 


In [14]:
#taking look at the test set
pred_rfc = pipeline_rf.predict(X_test)
accuracy = accuracy_score(y_test, pred_rfc)
print("Random forest TEST Accuracy:", accuracy)

pred_rfc = pipeline_lr.predict(X_test)
accuracy = accuracy_score(y_test, pred_rfc)
print("Logistic Regression TEST Accuracy:", accuracy)

Random forest TEST Accuracy: 0.9435736677115988
Logistic Regression TEST Accuracy: 0.8840125391849529


In [41]:
# Cell:11
# Artificial neural network

X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(X, y, test_size=0.15)

ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train_nn)
X_test_scaled = ss.transform(X_test_nn)


# OneHotEncoding Target Variable
y_train_encoded = pd.get_dummies(y_train_nn)
y_test_encoded = pd.get_dummies(y_test_nn)

# Code from https://www.kaggle.com/fatihdeniz/ann-fetal-health-classification
ann_classifier = keras.Sequential([
    # Input Layer
    keras.layers.Dense(20, input_shape=X_train_scaled[0].shape, activation="relu", kernel_initializer="HeNormal"),
    
    # Hidden Layer 1
    keras.layers.Dense(40,activation="relu"),
    keras.layers.Dense(80,activation="relu"),
    keras.layers.Dropout(0.3),
    
    # Hidden Layer 2
    keras.layers.Dense(160,activation="relu"),
    keras.layers.Dense(320,activation="relu"),
    keras.layers.Dropout(0.3),
    
    # Hidden Layer 3
    keras.layers.Dense(640,activation="relu"),
    keras.layers.Dense(3, activation="softmax")
])

ann_classifier.summary()


ann_classifier.compile(loss="categorical_crossentropy", metrics="accuracy", optimizer="rmsprop")

history = ann_classifier.fit(X_train_scaled, y_train_encoded, validation_data=(X_test_scaled, y_test_encoded), epochs=50)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_49 (Dense)            (None, 20)                280       
                                                                 
 dense_50 (Dense)            (None, 40)                840       
                                                                 
 dense_51 (Dense)            (None, 80)                3280      
                                                                 
 dropout_14 (Dropout)        (None, 80)                0         
                                                                 
 dense_52 (Dense)            (None, 160)               12960     
                                                                 
 dense_53 (Dense)            (None, 320)               51520     
                                                                 
 dropout_15 (Dropout)        (None, 320)              

In [38]:
y_pred_prob = ann_classifier.predict(X_test_scaled)
y_pred_prob

y_pred = []

for i in y_pred_prob:
    y_pred.append(np.argmax(i) + 1)
    
y_pred = np.array(y_pred)
y_pred

ann_accuracy = accuracy_score(y_pred, y_test_nn)
print("ANN Accuracy :", ann_accuracy)

ANN Accuracy : 0.9178403755868545


**<span style="color:#003f5c;"> If you liked this Notebook, please do upvote.</span>**

**<span style="color:#003f5c;"> If you have any suggestions or questions, I am all ears!</span>**

**<span style="color:#003f5c;">Best Wishes!</span>**

<a id="5"></a> 
# <h1 style='background:#f7b2b0; border:0; color:black'><center>END</center></h1> 